# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [12]:
# Importing the dataset

df=pd.read_csv("pokemon2_combotype.csv")
df.sample(5)

,#,name,type_1,type_2,total,hp,attack,defense,sp._atk,sp._def,speed,generation,legendary,a/d_ratio,combotype
592,532,Timburr,Fighting,NaN,305,75,80,55,25,35,35,5,False,1.454545,Fighting-
691,630,Mandibuzz,Dark,Flying,510,110,65,105,55,95,80,5,False,0.619048,Dark-Flying
75,69,Bellsprout,Grass,Poison,300,50,75,35,70,30,40,1,False,2.142857,Grass-Poison
382,350,Milotic,Water,NaN,540,95,60,79,100,125,81,3,False,0.759494,Water-
386,354,Banette,Ghost,NaN,455,64,115,65,83,63,65,3,False,1.769231,Ghost-


From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [13]:
# your code here
df.head()

,#,name,type_1,type_2,total,hp,attack,defense,sp._atk,sp._def,speed,generation,legendary,a/d_ratio,combotype
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,1.000000,Grass-Poison
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,0.984127,Grass-Poison
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,0.987952,Grass-Poison
3,3,Mega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False,0.813008,Grass-Poison
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False,1.209302,Fire-


In [16]:
(df.total==df.hp+df.attack+df.defense+df["sp._atk"]+df["sp._def"]+df.speed).value_counts()

True    800
Name: count, dtype: int64

## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

El problema del método siguiente es que nos crea dos juegos enteros de columnas "type_1" y "type_2", cuando solo queremos un juego completo de todos los tipos de pokemon y que, en aquellos que sean de dos tipos, aparezcan dos "unos" donde correspondan.

 your code here
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder(sparse_output=False).set_output(transform="pandas")
ohetransform=ohe.fit_transform(df[["type_1","type_2"]])
ohetransform.head(10)
df2=df.join(ohetransform)
df2

Así, usaremos otro método de sklearn, llamado "multilabel binarizer"

In [26]:
from sklearn.preprocessing import MultiLabelBinarizer
df["types_cobined"]=df[["type_1","type_2"]].apply(lambda x: [i for i in x if pd.notna(i)],axis=1) 

In [38]:
# Renombro porque me he equivocado al crear columna
df=df.rename(columns={"types_cobined":"types_combined"})

In [32]:
#Aquí creamos la codificación
mlb= MultiLabelBinarizer()
ohe_transform=mlb.fit_transform(df["types_combined"])
ohe_transform


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [ ]:
# Creamos un dataframe con los resultados
ohe_df=pd.DataFrame(ohe_transform,columns=mlb.classes_)
#Y lo unimos al dataframe de pokemon original
df2=pd.concat([df,ohe_df],axis=1)
df2

,#,name,type_1,type_2,total,hp,attack,defense,sp._atk,sp._def,...,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,...,0,1,0,0,0,1,0,0,0,0
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,...,0,1,0,0,0,1,0,0,0,0
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,...,0,1,0,0,0,1,0,0,0,0
3,3,Mega Venusaur,Grass,Poison,625,80,100,123,122,120,...,0,1,0,0,0,1,0,0,0,0
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,...,0,0,0,0,0,0,0,1,0,0
796,719,Mega Diancie,Rock,Fairy,700,50,160,110,160,110,...,0,0,0,0,0,0,0,1,0,0
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,...,1,0,0,0,0,0,1,0,0,0
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,...,0,0,0,0,0,0,1,0,0,0


o

## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [ ]:
# Para calcular la correlación:
#Calcular la correlación entre "Total" y cada columna de tipo de Pokemon
correlations=df2[["total"]+[col for col in df2.columns if col in ["Ghost", "Grass", "Fire", "Water", "Electric", "Ice", "Fighting", "Poison", "Ground", "Flying", "Psychic", "Bug", "Rock", "Dragon", "Dark", "Steel", "Fairy"]]].corr()["total"]


In [54]:
print(f"La correlación de total con cada tipo es\n {round(correlations,2)}")

La correlación de total con cada tipo es
 total       1.00
Bug        -0.15
Dark        0.06
Dragon      0.23
Electric    0.02
Fairy      -0.04
Fighting    0.08
Fire        0.08
Flying      0.06
Ghost       0.00
Grass      -0.05
Ground      0.02
Ice         0.06
Poison     -0.09
Psychic     0.12
Rock        0.03
Steel       0.11
Water      -0.02
Name: total, dtype: float64


Así pues, parece que el tipo de pokemon que más se correlaciona con valores más altos por lo general, es el tipo "Dragon", seguido del tipo "Psychic".

# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [58]:
# your code here
# Ahora nos quedaremos solo con los pokemon que sean de ambos tipos
df_two_types= df2.dropna(subset=["type_1","type_2"])

#Calculamos la correlación 
correl_two_types=df_two_types[["total"]+[col for col in df2.columns if col in["Ghost", "Grass", "Fire", "Water", "Electric", "Ice", "Fighting", "Poison", "Ground", "Flying", "Psychic", "Bug", "Rock", "Dragon", "Dark", "Steel", "Fairy"]]].corr()["total"]

# Filtramos solo las columnas de cada tipo y ordenamos de mayor a menor 
correl_two_types=correl_two_types.drop("total").sort_values(ascending=False)

correl_two_types

Dragon      0.324208
Fighting    0.143950
Fire        0.135961
Psychic     0.093013
Steel       0.083201
Electric    0.050108
Ice         0.046567
Dark        0.028288
Rock       -0.000605
Ground     -0.014493
Flying     -0.024826
Ghost      -0.045354
Water      -0.051025
Fairy      -0.071636
Grass      -0.108008
Poison     -0.157293
Bug        -0.159988
Name: total, dtype: float64

In [ ]:
En este caso elegiríamos tipo "Dragon" y tipo "Fighting".